<a href="https://colab.research.google.com/github/chanukaDissanayaka/RideFareClassification/blob/master/Ridefare_submission_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#imports

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from datetime import datetime, timedelta
from pandas import DataFrame

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

import numpy as np

RANDOM_SEED = 4

In [0]:
dataframe = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

dataframe.isna().sum(axis = 0)
#dataframe = dataframe.fillna(method='ffill')
dataframe = dataframe.dropna()

In [0]:
def preprocess_train_data(dataframe):
  feature_set = dataframe.iloc[:,1:13]

  features = feature_set

  features['pickup_time'] = pd.to_datetime(features['pickup_time'], format="%m/%d/%Y %H:%M")
  features['drop_time'] = pd.to_datetime(features['drop_time'], format="%m/%d/%Y %H:%M")

  td = features

  td['pickup_time_year'] = td['pickup_time'].dt.year
  td['pickup_time_month'] = td['pickup_time'].dt.month
  td['pickup_time_day'] = td['pickup_time'].dt.day
  td['pickup_time_hour'] = td['pickup_time'].dt.hour
  td['pickup_time_minute'] = td['pickup_time'].dt.minute

  td['drop_time_year'] = td['drop_time'].dt.year
  td['drop_time_month'] = td['drop_time'].dt.month
  td['drop_time_day'] = td['drop_time'].dt.day
  td['drop_time_hour'] = td['drop_time'].dt.hour
  td['drop_time_minute'] = td['drop_time'].dt.minute

  td = td.drop('pickup_time',1)
  td = td.drop('drop_time',1)

  return td

def get_labels(dataframe):
  label_set = dataframe.iloc[:, 13:14]

  rev_label_set = label_set.replace({'correct':'incorrect', 'incorrect':'correct'})

  encoder = LabelEncoder()
  encoder.fit(rev_label_set)
  labels = encoder.transform(rev_label_set)
  return labels

def set_time_of_the_day(cleaned_train_set):
  cleaned_train_set.head()
  newFrame = cleaned_train_set
  tt = newFrame

  d0_index = newFrame[(newFrame['pickup_time_hour']>=0) & (newFrame['pickup_time_hour'] < 6 )].index
  tt.loc[d0_index,'time_of_day']=0

  d1_index = newFrame[(newFrame['pickup_time_hour']>=6) & (newFrame['pickup_time_hour'] < 12 )].index
  tt.loc[d1_index,'time_of_day']=1

  d2_index = newFrame[(newFrame['pickup_time_hour']>=12) & (newFrame['pickup_time_hour'] < 18 )].index
  tt.loc[d2_index,'time_of_day']=2

  d3_index = newFrame[(newFrame['pickup_time_hour']>=18) & (newFrame['pickup_time_hour'] < 24 )].index
  tt.loc[d3_index,'time_of_day']=3

  return tt

In [5]:
cleaned_train_set = preprocess_train_data(dataframe)
labels = get_labels(dataframe)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
cleaned_train_set = cleaned_train_set.drop('pickup_time_year',1)
cleaned_train_set = cleaned_train_set.drop('pickup_time_month',1)
cleaned_train_set = cleaned_train_set.drop('pickup_time_day',1)
cleaned_train_set = cleaned_train_set.drop('pickup_time_minute',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_year',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_month',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_day',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_minute',1)
#cleaned_train_set = set_time_of_the_day(cleaned_train_set)

In [0]:
cleaned_train_set.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_hour,drop_time_hour
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,0,0
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,0,1
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,1,1
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,2,2
5,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,5,6


In [0]:
numeric_cols = ['meter_waiting','meter_waiting_fare','meter_waiting_till_pickup',
               'additional_fare','duration','pick_lat','pick_lon','drop_lat','drop_lon',
               'fare','pickup_time_day','pickup_time_hour','pickup_time_minute']
numeric_cols = ['meter_waiting','meter_waiting_fare','meter_waiting_till_pickup',
               'additional_fare','duration','pick_lat','pick_lon','drop_lat','drop_lon',
               'fare','pickup_time_hour']
#categorical_cols = ['time_of_day']

In [0]:
X_train, X_eval, Y_train, Y_eval = train_test_split(
    cleaned_train_set,
    labels,
    test_size=0.20,
    shuffle=True,
    stratify=labels,
    random_state=RANDOM_SEED
)

In [0]:
numeric_preprocessing_steps = Pipeline([
    ('minmax_scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])


preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols),
        #("categorical", categorical_transformer, categorical_cols)
    ],
    remainder = "drop"
)

In [0]:
data_pipeline = Pipeline([
    ("preprocessor", preprocessor)
])

In [0]:
data_pipeline.fit(X_train)
d_train = data_pipeline.transform(X_train)
d_eval = data_pipeline.transform(X_eval)

In [0]:
d_train.shape

(13574, 11)

In [0]:
d_train[0]

array([2.138e-04, 0.000e+00, 2.067e-03, 8.206e-04, 1.333e-03, 4.629e-01,
       2.780e-02, 2.283e-02, 9.676e-01, 5.276e-03, 4.783e-01])

In [0]:
from sklearn.utils.class_weight import compute_sample_weight
sample_w = compute_sample_weight(class_weight={0:5, 1:1}, y=Y_train)

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

gbclf = GradientBoostingClassifier(random_state=0, n_estimators = 800, max_depth =10, subsample = 1.0)
gbclf.fit(d_train, Y_train, sample_w)
gbclf.score(d_eval, Y_eval)

0.9490276959340012

In [0]:
cleaned_test_data = preprocess_train_data(test)

cleaned_test_data = cleaned_test_data.drop('pickup_time_year',1)
cleaned_test_data = cleaned_test_data.drop('pickup_time_month',1)
cleaned_test_data = cleaned_test_data.drop('pickup_time_day',1)
cleaned_test_data = cleaned_test_data.drop('pickup_time_minute',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_year',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_month',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_day',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_minute',1)

test_set = data_pipeline.transform(cleaned_test_data)

In [0]:
test_set.shape

(8576, 11)

In [0]:
preds = gbclf.predict(test_set)
print(preds)

[1 1 1 ... 1 1 1]


In [0]:
preds.shape

(8576,)

In [0]:
output = pd.DataFrame(test,columns=['tripid'])
output['prediction'] = preds
output.head()

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


In [0]:
output.to_csv('/content/Ridefare_submission_9.csv',index=False)

In [0]:
base = gbclf.predict(d_eval)
#print("keras", f1_score(Y_eval, eval_predict, average='micro'))
print("base", f1_score(Y_eval, base, average='micro'))

from sklearn.metrics import confusion_matrix

results = gbclf.predict(d_eval)
confusion_matrix(Y_eval, results)

base 0.9490276959340012


array([[ 170,  135],
       [  38, 3051]])

In [0]:
k =0
for i in pred_proba:
  print("%0.5f"%i[0],"%0.4f"%i[1], Y_eval[k])
  k = k+1

In [0]:
pred_proba = gbclf.predict_proba(d_eval)

threshold = 0.3

preds = (pred_proba [:,1] >= threshold).astype('int')

In [0]:
base = gbclf.predict(d_eval)
#print("keras", f1_score(Y_eval, eval_predict, average='micro'))
print("base", f1_score(Y_eval, base, average='micro'))

from sklearn.metrics import confusion_matrix

results = gbclf.predict(d_eval)
confusion_matrix(Y_eval, results)

In [0]:
pred_proba = gbclf.predict_proba(d_eval)

threshold = 0.05

preds = (pred_proba [:,0] <= threshold).astype('int')

In [0]:
print(preds)
print("base", f1_score(Y_eval, preds, average='micro'))
confusion_matrix(Y_eval, preds)

[1 0 1 ... 1 1 1]
base 0.940777843252799


array([[ 200,  105],
       [  96, 2993]])

In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0, class_weight={0:1, 1:8}, max_depth = 20)
clf.fit(d_train, Y_train)
clf.score(d_eval, Y_eval)

0.9281084266352386

In [0]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

test = SelectKBest(score_func=f_classif, k=17)
fit = test.fit(X_train, Y_train)
# summarize scores
set_printoptions(precision=3)
print(fit.scores_)
features_selected = fit.transform(cleaned_train_set)

k = cleaned_train_set.keys()
j =0
for i in fit.scores_:
  print (i, k[j])
  j = j+1

[2.560e+01 1.501e+02 1.365e+02 1.262e+02 1.618e+01 4.756e+00 1.069e+01
 4.670e-01 6.644e+00 4.090e+02 3.445e-01 1.521e-01 4.540e-01 2.967e-01
 3.661e-01 9.497e-05 1.896e-01]
25.601700516148178 additional_fare
150.1423482323858 duration
136.53814018608236 meter_waiting
126.15291654442362 meter_waiting_fare
16.176969168253855 meter_waiting_till_pickup
4.755913729479669 pick_lat
10.68640597221555 pick_lon
0.46699771908048066 drop_lat
6.644427453624097 drop_lon
409.0290439880105 fare
0.3444733818253683 pickup_time_day
0.15214708901589488 pickup_time_hour
0.45403506787917486 pickup_time_minute
0.2967370493315319 drop_time_day
0.3660508242748645 drop_time_hour
9.497351291964044e-05 drop_time_minute
0.18958475782896403 time_of_day
